## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,71.10,66,10000,17.76,clear sky
1,1,Saint Paul Harbor,US,57.7900,-152.4072,30.06,89,4760,10.18,overcast clouds
2,2,Carnarvon,AU,-24.8667,113.6333,77.07,83,10000,5.75,clear sky
3,3,Vaini,TO,-21.2000,-175.2000,82.18,80,10000,20.13,overcast clouds
4,4,Saldanha,ZA,-33.0117,17.9442,68.16,70,10000,24.02,clear sky


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Enter the minimum desirable temperature: '))
max_temp = float(input('Enter the maximum desirable temperature: '))

Enter the minimum desirable temperature: 70
Enter the maximum desirable temperature: 88


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_filtered_df = city_data_df.loc[(city_data_df['Max Temperature'] >= min_temp) &
                                    (city_data_df['Max Temperature'] <= max_temp)]
city_filtered_df.tail()

,Unnamed: 0,City,Country,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Current Description
695,695,Kantunil,MX,20.8000,-89.0333,75.92,87,10000,3.85,scattered clouds
703,703,Bintulu,MY,3.1667,113.0333,77.41,100,10000,0.00,scattered clouds
704,704,Trairi,BR,-3.2778,-39.2689,79.14,80,10000,5.93,light rain
705,705,Iquique,CL,-20.2208,-70.1431,71.64,67,10000,5.66,scattered clouds
708,708,Kaputa,ZM,-8.4689,29.6619,78.76,67,10000,4.52,light rain


In [23]:
# 4a. Determine if there are any empty rows.
city_filtered_df.isna().sum()

Unnamed: 0             0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temperature        0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_filtered_df.dropna()
clean_df.isna().count()

Unnamed: 0             251
City                   251
Country                251
Lat                    251
Lng                    251
Max Temperature        251
Humidity               251
Cloudiness             251
Wind Speed             251
Current Description    251
dtype: int64

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temperature", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temperature,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,71.10,clear sky,-33.6500,115.3333,
2,Carnarvon,AU,77.07,clear sky,-24.8667,113.6333,
3,Vaini,TO,82.18,overcast clouds,-21.2000,-175.2000,
5,Bengkulu,ID,79.43,broken clouds,-3.8004,102.2655,
6,San-Pédro,CI,86.88,clear sky,4.7485,-6.6363,
10,Paita,PE,73.96,overcast clouds,-5.0892,-81.1144,
12,Rikitea,PF,77.52,overcast clouds,-23.1203,-134.9692,
17,Cabedelo,BR,84.87,broken clouds,-6.9811,-34.8339,
20,Namatanai,PG,79.47,light rain,-3.6667,152.4333,
21,Morón,AR,71.92,clear sky,-34.6534,-58.6198,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    

    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

In [9]:
# 7. Drop the rows where there is no Hotel Name.


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
